In [1]:
import numpy as np
import names
import uuid
from cassandra.cluster import Cluster
from itertools import product, combinations

In [2]:
cities = ["Lublin", "Warszawa", "Poznan", "Krakow", "Gdansk","Radom","Wroclaw","Lodz","Katowice"]
first_names = ["John", "Marek", "Jerzy", "Klaus", "Wojtek", "Dawid", "Eustachy", "Diego", "Luther"]
second_names = ["Kenedy", "Obama", "Kowalski", "Trump", "Piekarski", "Szymczak", "Patyk", "Piaseczny", "Grzyb", "Kloc"]

In [3]:
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()

In [4]:
def create_keyspace(session):
    try:
        session.execute("DROP KEYSPACE IF EXISTS reservation_flights")
        session.execute("CREATE KEYSPACE IF NOT EXISTS reservation_flights WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }")
        session.execute("USE reservation_flights")
        print("done")
    except Exception as e:
        print("Execution error", e)
        
    return session

In [5]:
create_keyspace(session)

done


In [6]:
def create_user(session):
    #łączenie z bazą
    user_ids = []
    try:
        session.execute('DROP TABLE IF EXISTS user')
        session.execute('CREATE TABLE user (id uuid, first_name text, last_name text, PRIMARY KEY(id))')
        
        for person in product(first_names, second_names):
            first_name, last_name = person[0], person[1]
            user_id = uuid.uuid1()
            user_ids.append(user_id)
            cql = "INSERT INTO user (id, first_name, last_name) VALUES (%s, %s, %s)"
            vals = [user_id, first_name, last_name]
            session.execute(cql, vals)
        print("done") 
    except Exception as e:
        print("Execution error", e)
        
    return user_ids

In [7]:
user_ids = create_user(session)

done


In [8]:
planes_name = ["Galczynski", "Syrenka", "Balczewski", "Sokol", "Kalista", "Stoch"]

In [9]:
def create_plane(session):
    plane_ids = []
    try:
        session.execute('DROP TABLE IF EXISTS plane;')
        session.execute('CREATE TABLE plane (id uuid, name text, PRIMARY KEY(id));') #PO CO SEAT ID?
        
        cql = "INSERT INTO plane (id, name) VALUES (%s, %s);"
        for plane in range(6):
            plane_id = uuid.uuid1()
            plane_ids.append(plane_id)
            name = planes_name[plane] #Generator nazw samolotów
            vals = [plane_id, name]
            session.execute(cql, vals)
        print("done")
    except Exception as e:
        print("Execution error", e)
        
    return plane_ids

In [10]:
plane_ids = create_plane(session)

done


In [13]:
def create_airport(session):
    airport_ids = []
    try:
        session.execute('DROP TABLE IF EXISTS airport;')
        session.execute('CREATE TABLE airport (id uuid, name text, city text, country text, PRIMARY KEY(id));')
        
        cql = "INSERT INTO airport (id, name, city, country) VALUES (%s, %s, %s, %s);"
        for airport in ["Lublin", "Warszawa", "Poznan", "Krakow", "Gdansk","Radom","Wroclaw","Lodz","Katowice"]:
            airport_id = uuid.uuid1()
            airport_ids.append(airport_id)
            city_name = airport
            airport_name = city_name +"_airport"
            vals = [airport_id, airport_name, city_name, "Poland"]
            session.execute(cql, vals)
        print("done")
    except NameError:
        print("Execution error")
        
    return airport_ids

In [14]:
airport_ids = create_airport(session)

done


In [15]:
def create_flight(session):
    flight_ids = []
    try:
        session.execute('DROP TABLE IF EXISTS flight;')
        session.execute('CREATE TABLE flight (id uuid, plane_id uuid, departure_airport_id uuid, arrival_airport_id uuid, PRIMARY KEY(id));')
        
        cql = "INSERT INTO flight (id, plane_id, departure_airport_id, arrival_airport_id) VALUES (%s, %s, %s, %s);"
        for i, dep_arr in enumerate(combinations(airport_ids, 2)):
            flight_id = uuid.uuid1()
            flight_ids.append(flight_id)
            departure_airport_id, arrival_airport_id = dep_arr
            plane_id = plane_ids[i%len(plane_ids)]
            
            vals = [flight_id,departure_airport_id,departure_airport_id,plane_id]
            session.execute(cql, vals)
        print("done")
    except Exception as e:
        print("Execution error", e)
        
    return flight_ids

In [16]:
flight_ids = create_flight(session)

done


In [17]:
def create_seat(session):
    try:
        session.execute('DROP TABLE IF EXISTS seat;')
        session.execute('CREATE TABLE seat (plane_id uuid, row int, column int, is_vacant boolean, PRIMARY KEY((plane_id), row, column));')
        
        cql = "INSERT INTO seat (plane_id, row, column, is_vacant) VALUES (%(plane_id)s, %(row)s, %(column)s, %(is_vacant)s);"
        for plane_id in plane_ids:
            for row in range(10):
                for col in range(6):
                    vals = {
                        "plane_id": plane_id,
                        "row": row,
                        "column": col,
                        "is_vacant": True
                    }
                    session.execute(cql, vals)
        print("done")

    except Exception as e:
        print("Execution error", e)

In [18]:
create_seat(session)

done


In [22]:
def create_reservation(session):
    try:
        session.execute('DROP TABLE IF EXISTS reservation;')
        session.execute('CREATE TABLE reservation (id uuid, user_id uuid, flight_id uuid, seat_id uuid, seat_row int, seat_col int, created_at timestamp, PRIMARY KEY(id));')
        print("done")

    except Exception as e:
        print("Execution error", e)

In [23]:
create_reservation(session)

done
